In [4]:
from __future__ import print_function, division
import scipy

from keras.datasets import cifar10
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from PIL import Image
import matplotlib.pyplot as plt

import sys
import os

import numpy as np

class COGAN():
    """Reference: https://wiseodd.github.io/techblog/2017/02/18/coupled_gan/"""
    def __init__(self):
        self.img_rows = 32
        self.img_cols = 32
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.d1, self.d2 = self.build_discriminators()
        self.d1.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])
        self.d2.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.g1, self.g2 = self.build_generators()

        # The generator takes noise as input and generated imgs
        z = Input(shape=(self.latent_dim,))
        img1 = self.g1(z)
        img2 = self.g2(z)

        # For the combined model we will only train the generators
        self.d1.trainable = False
        self.d2.trainable = False

        # The valid takes generated images as input and determines validity
        valid1 = self.d1(img1)
        valid2 = self.d2(img2)

        # The combined model  (stacked generators and discriminators)
        # Trains generators to fool discriminators
        self.combined = Model(z, [valid1, valid2])
        self.combined.compile(loss=['binary_crossentropy', 'binary_crossentropy'],
                                    optimizer=optimizer)

    def build_generators(self):

        # Shared weights between generators
        model = Sequential()
        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))

        noise = Input(shape=(self.latent_dim,))
        feature_repr = model(noise)

        # Generator 1
        g1 = Dense(1024)(feature_repr)
        g1 = LeakyReLU(alpha=0.2)(g1)
        g1 = BatchNormalization(momentum=0.8)(g1)
        g1 = Dense(np.prod(self.img_shape), activation='tanh')(g1)
        img1 = Reshape(self.img_shape)(g1)

        # Generator 2
        g2 = Dense(1024)(feature_repr)
        g2 = LeakyReLU(alpha=0.2)(g2)
        g2 = BatchNormalization(momentum=0.8)(g2)
        g2 = Dense(np.prod(self.img_shape), activation='tanh')(g2)
        img2 = Reshape(self.img_shape)(g2)

        model.summary()

        return Model(noise, img1), Model(noise, img2)

    def build_discriminators(self):

        img1 = Input(shape=self.img_shape)
        img2 = Input(shape=self.img_shape)

        # Shared discriminator layers
        model = Sequential()
        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))

        img1_embedding = model(img1)
        img2_embedding = model(img2)

        # Discriminator 1
        validity1 = Dense(1, activation='sigmoid')(img1_embedding)
        # Discriminator 2
        validity2 = Dense(1, activation='sigmoid')(img2_embedding)

        return Model(img1, validity1), Model(img2, validity2)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = cifar10.load_data()

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        # X_train = np.expand_dims(X_train, axis=3)

        # Images in domain A and B (rotated)
        X1 = X_train[:int(X_train.shape[0]//2)]
        X2 = X_train[int(X_train.shape[0]//2):]
        X2 = scipy.ndimage.interpolation.rotate(X2, 90, axes=(1, 2))

        

        nb_batches = int(X_train.shape[0] // batch_size)
        global_step = 0

        for epoch in range(epochs):

            for index in range(nb_batches):
                global_step += 1
                # imgs = X_train[index * batch_size:(index + 1) * batch_size]

                imgs1 = X1[index * batch_size:(index + 1) * batch_size]
                imgs2 = X2[index * batch_size:(index + 1) * batch_size]
                # Adversarial ground truths
                valid = np.ones((len(imgs1), 1))
                fake = np.zeros((len(imgs2), 1))

                # Sample noise as generator input
                noise = np.random.normal(0, 1, (len(imgs1), 100))

                # Generate a batch of new images
                gen_imgs1 = self.g1.predict(noise)
                gen_imgs2 = self.g2.predict(noise)

                # Train the discriminators
                d1_loss_real = self.d1.train_on_batch(imgs1, valid)
                d2_loss_real = self.d2.train_on_batch(imgs2, valid)
                d1_loss_fake = self.d1.train_on_batch(gen_imgs1, fake)
                d2_loss_fake = self.d2.train_on_batch(gen_imgs2, fake)
                d1_loss = 0.5 * np.add(d1_loss_real, d1_loss_fake)
                d2_loss = 0.5 * np.add(d2_loss_real, d2_loss_fake)

                # ------------------
                #  Train Generators
                # ------------------

                g_loss = self.combined.train_on_batch(noise, [valid, valid])

                # Plot the progress
                print("epoch:%d step:%d [D1 loss: %f, acc.: %.2f%%] [D2 loss: %f, acc.: %.2f%%] [G loss: %f]" \
                      % (epoch, global_step, d1_loss[0], 100 * d1_loss[1], d2_loss[0], 100 * d2_loss[1], g_loss[0]))

                # If at save interval => save generated image samples
                if  global_step % sample_interval == 0:
                    self.sample_images(epoch, global_step)

    def sample_images(self, epoch,global_step):
        r, c = 10, 10
        noise = np.random.normal(0, 1, (r * int(c//2), 100))
        gen_imgs1 = self.g1.predict(noise)
        gen_imgs2 = self.g2.predict(noise)

        generated_images = np.concatenate([gen_imgs1, gen_imgs2])

        generated_images = np.asarray((generated_images * 127.5 + 127.5).astype(np.uint8))

        def vis_square(data, padsize=1, padval=0):
            # force the number of filters to be square
            n = int(np.ceil(np.sqrt(data.shape[0])))
            padding = ((0, n ** 2 - data.shape[0]), (0, padsize), (0, padsize)) + ((0, 0),) * (data.ndim - 3)
            data = np.pad(data, padding, mode='constant', constant_values=(padval, padval))

            # tile the filters into an image
            data = data.reshape((n, n) + data.shape[1:]).transpose((0, 2, 1, 3) + tuple(range(4, data.ndim + 1)))
            data = data.reshape((n * data.shape[1], n * data.shape[3]) + data.shape[4:])
            return data

        img = vis_square(generated_images)
        if not os.path.isdir('images_cogan_cifar10'):
            os.mkdir('images_cogan_cifar10')
        Image.fromarray(img).save(
            "images_cogan_cifar10/epoch_%d_step_%d.png" % (epoch, global_step))


if __name__ == '__main__':
    gan = COGAN()
    gan.train(epochs=50, batch_size=64, sample_interval=200)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_35 (Dense)             (None, 256)               25856     
_________________________________________________________________
leaky_re_lu_21 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
batch_normalization_13 (Batc (None, 256)               1024      
_________________________________________________________________
dense_36 (Dense)             (None, 512)               131584    
_________________________________________________________________
leaky_re_lu_22 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
batch_normalization_14 (Batc (None, 512)               2048      
Total params: 160,512
Trainable params: 158,976
Non-trainable params: 1,536
_________________________________________________________________


epoch:0 step:72 [D1 loss: 0.738930, acc.: 57.81%] [D2 loss: 0.791218, acc.: 51.56%] [G loss: 3.971403]
epoch:0 step:73 [D1 loss: 0.767198, acc.: 51.56%] [D2 loss: 0.691995, acc.: 65.62%] [G loss: 3.625148]
epoch:0 step:74 [D1 loss: 0.730717, acc.: 61.72%] [D2 loss: 0.581367, acc.: 68.75%] [G loss: 4.029885]
epoch:0 step:75 [D1 loss: 0.731491, acc.: 60.94%] [D2 loss: 0.780753, acc.: 60.94%] [G loss: 4.015647]
epoch:0 step:76 [D1 loss: 0.740478, acc.: 63.28%] [D2 loss: 0.641445, acc.: 71.09%] [G loss: 3.761047]
epoch:0 step:77 [D1 loss: 0.759484, acc.: 53.91%] [D2 loss: 0.603537, acc.: 67.97%] [G loss: 4.138339]
epoch:0 step:78 [D1 loss: 0.697549, acc.: 65.62%] [D2 loss: 0.734927, acc.: 67.19%] [G loss: 4.393838]
epoch:0 step:79 [D1 loss: 1.108809, acc.: 42.97%] [D2 loss: 0.730742, acc.: 63.28%] [G loss: 3.633979]
epoch:0 step:80 [D1 loss: 0.753716, acc.: 60.16%] [D2 loss: 0.657848, acc.: 63.28%] [G loss: 3.617743]
epoch:0 step:81 [D1 loss: 0.791153, acc.: 57.03%] [D2 loss: 0.763648, acc

epoch:0 step:153 [D1 loss: 0.715175, acc.: 54.69%] [D2 loss: 0.621574, acc.: 71.09%] [G loss: 3.485997]
epoch:0 step:154 [D1 loss: 0.752877, acc.: 52.34%] [D2 loss: 0.625656, acc.: 67.19%] [G loss: 3.250177]
epoch:0 step:155 [D1 loss: 0.658243, acc.: 60.94%] [D2 loss: 0.611460, acc.: 66.41%] [G loss: 3.381281]
epoch:0 step:156 [D1 loss: 0.706044, acc.: 60.94%] [D2 loss: 0.542375, acc.: 68.75%] [G loss: 3.337784]
epoch:0 step:157 [D1 loss: 0.741575, acc.: 55.47%] [D2 loss: 0.706682, acc.: 57.03%] [G loss: 3.591253]
epoch:0 step:158 [D1 loss: 0.693994, acc.: 54.69%] [D2 loss: 0.645383, acc.: 64.06%] [G loss: 3.389635]
epoch:0 step:159 [D1 loss: 0.733816, acc.: 56.25%] [D2 loss: 0.583495, acc.: 69.53%] [G loss: 3.631968]
epoch:0 step:160 [D1 loss: 0.676784, acc.: 60.94%] [D2 loss: 0.580536, acc.: 67.97%] [G loss: 3.574291]
epoch:0 step:161 [D1 loss: 0.754097, acc.: 56.25%] [D2 loss: 0.668106, acc.: 65.62%] [G loss: 3.702013]
epoch:0 step:162 [D1 loss: 0.676740, acc.: 64.06%] [D2 loss: 0.7

epoch:0 step:233 [D1 loss: 0.706380, acc.: 53.12%] [D2 loss: 0.561797, acc.: 74.22%] [G loss: 3.654655]
epoch:0 step:234 [D1 loss: 0.685467, acc.: 62.50%] [D2 loss: 0.548635, acc.: 74.22%] [G loss: 3.222407]
epoch:0 step:235 [D1 loss: 0.614432, acc.: 68.75%] [D2 loss: 0.569089, acc.: 70.31%] [G loss: 3.198163]
epoch:0 step:236 [D1 loss: 0.565462, acc.: 69.53%] [D2 loss: 0.551756, acc.: 72.66%] [G loss: 3.425713]
epoch:0 step:237 [D1 loss: 0.615429, acc.: 67.97%] [D2 loss: 0.601173, acc.: 72.66%] [G loss: 3.139544]
epoch:0 step:238 [D1 loss: 0.670134, acc.: 65.62%] [D2 loss: 0.531264, acc.: 71.09%] [G loss: 3.058449]
epoch:0 step:239 [D1 loss: 0.686562, acc.: 62.50%] [D2 loss: 0.534818, acc.: 73.44%] [G loss: 3.313481]
epoch:0 step:240 [D1 loss: 0.808764, acc.: 48.44%] [D2 loss: 0.558301, acc.: 71.09%] [G loss: 3.651941]
epoch:0 step:241 [D1 loss: 0.746781, acc.: 54.69%] [D2 loss: 0.519529, acc.: 71.88%] [G loss: 3.525084]
epoch:0 step:242 [D1 loss: 0.669733, acc.: 60.16%] [D2 loss: 0.5

epoch:0 step:313 [D1 loss: 0.870607, acc.: 57.03%] [D2 loss: 0.532602, acc.: 81.25%] [G loss: 3.441453]
epoch:0 step:314 [D1 loss: 0.778739, acc.: 49.22%] [D2 loss: 0.540202, acc.: 78.12%] [G loss: 3.280724]
epoch:0 step:315 [D1 loss: 0.709105, acc.: 47.66%] [D2 loss: 0.592575, acc.: 69.53%] [G loss: 3.027941]
epoch:0 step:316 [D1 loss: 0.723248, acc.: 51.56%] [D2 loss: 0.626860, acc.: 62.50%] [G loss: 2.921394]
epoch:0 step:317 [D1 loss: 0.650515, acc.: 67.19%] [D2 loss: 0.679467, acc.: 56.25%] [G loss: 2.886636]
epoch:0 step:318 [D1 loss: 0.680366, acc.: 58.59%] [D2 loss: 0.602189, acc.: 64.06%] [G loss: 3.326323]
epoch:0 step:319 [D1 loss: 0.728918, acc.: 53.12%] [D2 loss: 0.598242, acc.: 64.84%] [G loss: 3.176640]
epoch:0 step:320 [D1 loss: 0.697174, acc.: 57.03%] [D2 loss: 0.609131, acc.: 66.41%] [G loss: 3.126348]
epoch:0 step:321 [D1 loss: 0.624062, acc.: 64.84%] [D2 loss: 0.570539, acc.: 71.88%] [G loss: 3.298130]
epoch:0 step:322 [D1 loss: 0.665827, acc.: 65.62%] [D2 loss: 0.5

IndexError: list index out of range